In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
STORE="Store 312"

In [3]:
sku_master = pd.read_excel("asset-v1_MITx+CTL.SC4x+3T2018+type@asset+block@TinyCo_Data_RAW.xlsx")
store = pd.read_excel("asset-v1_MITx+CTL.SC4x+3T2018+type@asset+block@TinyCo_Data_RAW.xlsx",
                      STORE)

In [4]:
sku_master

,Department,Class,Style,Color,SKU,Cost,Price,Vendor
0,800,5,20,2,8000520021,7.5,9.99,MA Excellent Products
1,800,4,51,11,8000451112,9.0,12.99,MA Excellent Products
2,731,24,55,52,7312455520,25.0,31.99,MA Excellent Products
3,731,24,55,53,7312455530,14.5,22.99,GA General Wholesales
4,5001,201,12,4,50012011240,2.5,7.99,China Imports
5,5001,201,12,5,50012011250,7.5,9.99,China Imports
6,5001,201,13,4,50012011341,8.0,6.99,China Imports
7,5001,300,1,1,50013000110,6.5,12.99,China Imports


In [5]:
store.head()

,DB_ID,SKU,Store,Date,Unit Sales,Dollar Sales
0,79444,50012011250,312,2014-08-04,3,29.97
1,79445,50012011250,312,2014-08-05,2,19.98
2,79446,50012011250,312,2014-08-06,5,49.95
3,79447,50012011250,312,2014-08-07,3,29.97
4,79448,50012011250,312,2014-08-08,7,69.93


Differently from a spreadsheet, I cannot compute summary statistics of columns which have non-numerical values. So I have to remove those first.

In [6]:
store = store[ store["Unit Sales"].apply(lambda x: type(x) == int or type(x) == float) ]
store["Unit Sales"]=store["Unit Sales"].astype(np.float)

In [7]:
store[["Unit Sales","Dollar Sales"]].describe()

,Unit Sales,Dollar Sales
count,3616.000000,3616.000000
mean,108.081305,1097.783678
std,2040.948271,20388.315627
min,-6.000000,-59.940000
25%,2.000000,15.980000
50%,3.000000,31.990000
75%,5.000000,68.970000
max,60000.000000,599400.000000


In [8]:
len(store)

3666

Removing zero-value records

In [9]:
store = store[ ( store[["Unit Sales","Dollar Sales"]] != 0 ).any(axis=1) ]

In [10]:
len(store)

3632

Remove Null records

In [11]:
store = store[ ( store.notnull() ).all(axis=1) ]

In [12]:
len(store)

3582

In [13]:
store.head()

,DB_ID,SKU,Store,Date,Unit Sales,Dollar Sales
0,79444,50012011250,312,2014-08-04,3.0,29.97
1,79445,50012011250,312,2014-08-05,2.0,19.98
2,79446,50012011250,312,2014-08-06,5.0,49.95
3,79447,50012011250,312,2014-08-07,3.0,29.97
4,79448,50012011250,312,2014-08-08,7.0,69.93


Rescaled the outliers. This is a bit *ad hoc*, and I should have looked for outliers instead of using a hardwired number (in this case 10000).

In [14]:
len(store[store["Unit Sales"] >= 10000])

11

In [15]:
store.loc[ store["Unit Sales"] >= 10000, ["Unit Sales","Dollar Sales"] ] = \
  store.loc[ store["Unit Sales"] >= 10000, ["Unit Sales","Dollar Sales"] ]/10000 

remove duplicates

In [17]:
store = store.drop_duplicates()

In [18]:
len(store)

3579

Show the records that do not have a corresponding entry on `sku_master`

In [19]:
store[~store.SKU.isin(sku_master.SKU)]

,DB_ID,SKU,Store,Date,Unit Sales,Dollar Sales
164,79608,50*12011250,312,2015-02-14,1.0,9.99
165,79609,50*12011250,312,2015-02-16,4.0,39.96
166,79610,50*12011250,312,2015-02-17,3.0,29.97
250,79694,50*12011250,312,2015-05-28,1.0,9.99
300,79744,50*12011250,312,2015-07-27,5.0,49.95
...,...,...,...,...,...,...
958,82858,50012011340,312,2015-12-26,1.0,4.99
959,82859,50012011340,312,2015-12-28,3.0,14.97
960,82860,50012011340,312,2015-12-29,3.0,14.97
961,82861,50012011340,312,2015-12-30,2.0,9.98


In [20]:
store[["Unit Sales","Dollar Sales"]].describe()

,Unit Sales,Dollar Sales
count,3579.000000,3579.000000
mean,5.825929,76.443557
std,9.044461,119.245469
min,-6.000000,-59.940000
25%,2.000000,15.980000
50%,3.000000,31.990000
75%,5.000000,68.970000
max,42.000000,699.930000
